In [ ]:
""" import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords


# Load the data
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\extracted_data.csv")

# Load pre-trained multilingual BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')

# Load Norwegian stopwords from NLTK
norwegian_stopwords = set(stopwords.words('norwegian'))

# Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in norwegian_stopwords]
    return ' '.join(filtered_words)

# Function to generate embeddings for a given text
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Apply stopword removal to the content
df['content'] = df['content'].apply(remove_stopwords)

# Generate embeddings for each tender content
df['embeddings'] = df['content'].apply(lambda x: generate_embeddings(x))

# Function to handle search query and find the most relevant tenders
def recommend_tenders(query, df):
    query = remove_stopwords(query)  # Remove stopwords from the query as well
    query_embedding = generate_embeddings(query)
    
    # Compute similarity scores between the query and each document
    similarities = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    
    # Sort tenders based on similarity and return the top 5 recommendations
    top_recommendations = df.iloc[similarities.argsort()[-5:]]
    return top_recommendations[['filename', 'source_format', 'content']]

# Example usage
query = "cloud computing"
recommended_tenders = recommend_tenders(query, df)
print(recommended_tenders)
 """

In [ ]:
""" import gradio as gr
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
df = pd.read_csv("C:/Users/spide/OneDrive/Desktop/Bachlorz/extracted_data.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate embeddings for a given text
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Generate embeddings for each tender content
df['embeddings'] = df['content'].apply(lambda x: generate_embeddings(x))

# Function to handle search query and find the most relevant tenders
def recommend_tenders(query, df):
    query_embedding = generate_embeddings(query)
    
    # Compute similarity scores between the query and each document
    similarities = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    
    # Sort tenders based on similarity and return the top 5 recommendations
    top_recommendations = df.iloc[similarities.argsort()[-5:]]
    
    # Add cosine similarity score to the result
    top_recommendations['cosine_similarity'] = similarities[similarities.argsort()[-5:]]
    
    return top_recommendations[['filename', 'source_format', 'content', 'cosine_similarity']]

# Gradio function for displaying recommendations
def recommend_tenders_gradio(query):
    recommended_tenders = recommend_tenders(query, df)
    return recommended_tenders

# Create Gradio Interface
interface = gr.Interface(fn=recommend_tenders_gradio, inputs="text", outputs="dataframe", title="Tender Recommendation System", description="Enter a search query to find the most relevant tenders.")

# Launch the interface
interface.launch()
 """

In [ ]:
""" # Gradio function for displaying recommendations
def recommend_tenders_gradio(query):
    recommended_tenders = recommend_tenders(query, df)
    return recommended_tenders

# Create Gradio Interface
interface = gr.Interface(fn=recommend_tenders_gradio, inputs="text", outputs="dataframe", live=True, title="Tender Recommendation System", description="Enter a search query to find the most relevant tenders.")

# Launch the interface
interface.launch() """


In [ ]:
""" query = "cloud computing krav" 
query_embedding = generate_embeddings(query)

# Compare similarity
similarity = cosine_similarity([query_embedding])
print("Cosine Similarity:", similarity[0][0])  # Expect a value between 0 and 1 """


In [ ]:
import pandas as pd
import re
from transformers import BertModel, BertTokenizer
import torch

# Step 1: Load the CSV file into the DataFrame
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\Data\extracted_data.csv")

# Step 2: Text Cleanup Function (removes unwanted characters, excessive spaces)
def clean_text(text):
    """
    This function cleans the extracted text by removing unwanted characters,
    excessive white spaces, and special characters.
    """
    # Remove excessive white spaces (replace multiple spaces with one)
    text = re.sub(r'\s+', ' ', text)  
    
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)  
    
    # Remove leading/trailing spaces
    text = text.strip()  
    
    return text

# Step 3: BERT Tokenizer and Model Setup (using a multilingual version of BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    """
    This function takes cleaned text as input, tokenizes it using BERT's tokenizer,
    and returns the contextual embeddings for the text.
    """
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings
    return embeddings

# Step 4: Clean the text in the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)

# Step 5: Generate BERT embeddings for the cleaned content
df['bert_embeddings'] = df['cleaned_content'].apply(get_bert_embeddings)

# Check the results
print(df[['content', 'cleaned_content', 'bert_embeddings']].head())

# Optional: Save the DataFrame with embeddings back to a new CSV
df.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\extracted_data_with_embeddings.csv", index=False)


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Load the dataset
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\Data\extracted_data.csv")  # Correct file path

# Define stopwords for both English and Norwegian
stop_words = set(stopwords.words('english')) | set(stopwords.words('norwegian'))

# Preprocess the content (cleaning text, removing stopwords)
def preprocess_text(text):
    tokens = text.split()  # Tokenize the text by splitting on spaces
    cleaned_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and len(token) > 2]  # Remove stopwords and short tokens
    return ' '.join(cleaned_tokens)

# Apply preprocessing to the 'content' column
df['cleaned_content'] = df['content'].apply(preprocess_text)

# Save the cleaned data to a new CSV file
df.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\cleaned_data.csv", index=False)

print("Stopword removal completed and saved to 'cleaned_data.csv'.")


In [4]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the multilingual BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Load the cleaned dataset
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\cleaned_data.csv")  # Correct path

# Function to get BERT embeddings
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling to get sentence representation
    return embeddings

# Apply BERT to get embeddings for all documents
embeddings = []
for text in df['cleaned_content']:
    embeddings.append(get_bert_embeddings(text).numpy())

# Convert list of embeddings to a numpy array
embeddings = np.vstack(embeddings)

# Standardize embeddings (important for clustering)
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)

# Dimensionality reduction (PCA)
pca = PCA(n_components=50)  # Reduce to 50 dimensions for better clustering
embeddings_pca = pca.fit_transform(embeddings_scaled)

# Perform K-Means clustering
n_topics = 10  # Set the number of topics
kmeans = KMeans(n_clusters=n_topics, random_state=42)
df['Topic'] = kmeans.fit_predict(embeddings_pca)

# Show the resulting topics with their top documents
for topic_num in range(n_topics):
    topic_docs = df[df['Topic'] == topic_num]
    print(f"Topic {topic_num}:")
    print(topic_docs['cleaned_content'].head())  # Display top documents in each topic
    print("\n")

# Optional: Save the topics to a CSV
df.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_bert.csv", index=False)

print("BERT-based topic modeling completed and saved to 'topics_with_bert.csv'.")


Topic 0:
494    february 2024 tender specifications part eeas/...
499    annex tender specifications invitation tender ...
502    april 2023 tender specifications call tenders ...
505    august 2023 tender specifications call tenders...
506    ref. ares(2023)7963335 22/11/2023 eurohpc-pn-2...
Name: cleaned_content, dtype: object


Topic 1:
2     vedlegg ii.01 bilag seriøsitetskrav bygg- anle...
5     kravspesifikasjon innledning kravspesifikasjon...
7     konkurransegrunnlag dokumentplan riving saneri...
8     trondheim eiendom avdeling utbygging b.5.2 mil...
10    kravspesifikasjon kravspesifikasjon revised 6/...
Name: cleaned_content, dtype: object


Topic 2:
495    european chemicals agency call tenders echa/20...
498    european commission directorate-general region...
500    european commission directorate-general commun...
503    ref. ares(2024)8902087 12/12/2024 european hig...
504    european fisheries control agency (efca) europ...
Name: cleaned_content, dtype: object


Topic 

In [14]:
import pandas as pd

# Load the CSV file (update the path if needed)
file_path = r'C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_bert.csv'
df = pd.read_csv(file_path)

# Check the first few rows of the dataframe to ensure it's loaded correctly
print(df.head())

# Group by topics and display the top documents for each topic
topic_groups = df.groupby('Topic')

# Show the top 5 documents for each topic
for topic_num, group in topic_groups:
    print(f"\nTopic {topic_num}:")
    top_docs = group['cleaned_content'].head()  # Show the top 5 documents for the topic
    for doc in top_docs:
        print(f"- {doc[:500]}...")  # Print the first 500 characters for brevity
    print("\n")


                                            filename source_format  \
0  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
1  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          xlsx   
2  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
3  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
4  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   

                                             content  \
0  Nedre Romerike brann- og redningsvesen IKS Rev...   
1  Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Un...   
2  Vedlegg II.01 Bilag 2 - Seriøsitetskrav for by...   
3  MAL – Retningslinjer og krav til prøvedrift PR...   
4  KONKURRANSEGRUNNLAG ÅPEN ANBUDSKONKURRANSE ett...   

                                     cleaned_content  Topic  
0  nedre romerike brann- redningsvesen iks revise...      8  
1  unnamed: unnamed: unnamed: unnamed: unnamed: u...      4  
2  vedlegg ii.01 bilag seriøsitetskrav bygg- anle...    

In [15]:
# Drop rows where 'cleaned_content' contains "unnamed" or NaN
df_cleaned = df[~df['cleaned_content'].str.contains("unnamed", case=False, na=False)]
df_cleaned = df_cleaned.dropna(subset=['cleaned_content'])  # Drop NaN rows in 'cleaned_content'

# Verify the cleaned dataframe
print(f"Cleaned DataFrame shape: {df_cleaned.shape}")
print(df_cleaned.head())

# Optionally, save the cleaned dataset to a new CSV
df_cleaned.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_bert_cleaned.csv", index=False)


Cleaned DataFrame shape: (1392, 5)
                                            filename source_format  \
0  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
2  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
3  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
4  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
5  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   

                                             content  \
0  Nedre Romerike brann- og redningsvesen IKS Rev...   
2  Vedlegg II.01 Bilag 2 - Seriøsitetskrav for by...   
3  MAL – Retningslinjer og krav til prøvedrift PR...   
4  KONKURRANSEGRUNNLAG ÅPEN ANBUDSKONKURRANSE ett...   
5  KRAVSPESIFIKASJON 1. INNLEDNING Kravspesifikas...   

                                     cleaned_content  Topic  
0  nedre romerike brann- redningsvesen iks revise...      8  
2  vedlegg ii.01 bilag seriøsitetskrav bygg- anle...      1  
3  mal retningslinjer

In [16]:
# Group by topics and display the top documents for each topic
topic_groups = df_cleaned.groupby('Topic')

# Show the top 5 documents for each topic
for topic_num, group in topic_groups:
    print(f"\nTopic {topic_num}:")
    top_docs = group['cleaned_content'].head()  # Show the top 5 documents for the topic
    for doc in top_docs:
        print(f"- {doc[:500]}...")  # Print the first 500 characters for brevity
    print("\n")



Topic 0:
- february 2024 tender specifications part eeas/deltlsd/2024/cpn/0145 security services delegation european union timor- leste lot security services delegation timor-leste lot technical safety security systems delegation timor-leste competitive procedure negotiation important information!! due confidential nature, technical specifications disclosed candidates successful end first stage procedure. submit technical financial tenders first stage. details, please see point document. table contents int...
- annex tender specifications invitation tender no. acer/op/igr/23/2023 annex report benefit-based remuneration efficient infrastructure investments european union agency cooperation energy regulators direct service contract tender specifications open call tenders acer/op/igr/23/2023 annex tender specifications invitation tender no. acer/op/igr/23/2023 table contents title invitation tender ........................................................................... background inf

In [18]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Function to get BERT embeddings for a document
def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Compute BERT embeddings for each document in the cleaned dataset
df_cleaned['bert_embeddings'] = df_cleaned['cleaned_content'].apply(lambda x: get_bert_embedding(x, tokenizer, model))

# Optionally, save the dataframe with embeddings
df_cleaned.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_bert_embeddings.csv", index=False)


In [19]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Function to compute embeddings for a batch of texts
def get_bert_embeddings_batch(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Set batch size
batch_size = 100  # Adjust depending on memory and size of dataset

# Load the cleaned dataset
df_cleaned = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\cleaned_data.csv")  # Update the path if necessary

# Initialize an empty list to store the embeddings
embeddings_list = []

# Batch process the dataset
for start in range(0, len(df_cleaned), batch_size):
    end = min(start + batch_size, len(df_cleaned))
    
    # Get the current batch of cleaned content
    batch = df_cleaned['cleaned_content'].iloc[start:end].tolist()
    
    # Compute embeddings for the current batch
    batch_embeddings = get_bert_embeddings_batch(batch, tokenizer, model)
    
    # Append the embeddings for the current batch to the embeddings list
    embeddings_list.append(batch_embeddings)

# Combine all the embeddings into a single tensor
document_embeddings = torch.cat(embeddings_list, dim=0)

# Optionally, you can save these embeddings to a CSV or other formats (e.g., pickle, npy)
# Saving embeddings as numpy array
np.save('document_embeddings.npy', document_embeddings.numpy())

# If you want to update your dataframe with the embeddings:
df_cleaned['bert_embeddings'] = list(document_embeddings.numpy())

# Save the dataframe with embeddings to a new CSV (optional)
df_cleaned.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_embeddings.csv", index=False)


In [22]:
print(df.columns)  # This will display the column names in the DataFrame
print(df.head())   # This will show the first few rows of the DataFrame


Index(['filename', 'source_format', 'content', 'cleaned_content', 'Topic'], dtype='object')
                                            filename source_format  \
0  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
1  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          xlsx   
2  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
3  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   
4  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...           pdf   

                                             content  \
0  Nedre Romerike brann- og redningsvesen IKS Rev...   
1  Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Un...   
2  Vedlegg II.01 Bilag 2 - Seriøsitetskrav for by...   
3  MAL – Retningslinjer og krav til prøvedrift PR...   
4  KONKURRANSEGRUNNLAG ÅPEN ANBUDSKONKURRANSE ett...   

                                     cleaned_content  Topic  
0  nedre romerike brann- redningsvesen iks revise...      8  
1  unnamed: unnamed: unnam

In [28]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset with embeddings
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_embeddings.csv")

# Ensure the embeddings are in proper format (i.e., as numpy arrays or lists)
def parse_embeddings(embedding_str):
    # If the embeddings are stored as numpy array strings, convert them back to numpy arrays
    return torch.tensor(np.fromstring(embedding_str.strip('[]'), sep=' '))

# Apply this function to convert embeddings from string to tensor
df['bert_embeddings'] = df['bert_embeddings'].apply(parse_embeddings)

# Initialize BERT tokenizer and model for query embedding
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Function to generate BERT embedding for a query
def get_query_embedding(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state.mean(dim=1)  # Average pooling
    return query_embedding.squeeze().numpy()  # Convert to numpy array for cosine similarity

# Function to recommend top N most similar documents based on a query
def recommend_documents(query, df, tokenizer, model, top_n=5):
    # Generate the query embedding
    query_embedding = get_query_embedding(query, tokenizer, model)

    # Get document embeddings (assuming they are already in 'bert_embeddings' column)
    # Convert each embedding back to a PyTorch tensor, then stack them
    document_embeddings = torch.stack([torch.tensor(embedding) for embedding in df['bert_embeddings'].values])

    # Calculate cosine similarity between query embedding and document embeddings
    similarities = cosine_similarity([query_embedding], document_embeddings.numpy())

    # Get the indices of the top N most similar documents
    top_indices = similarities.argsort()[0][-top_n:][::-1]

    # Get the top N documents and their similarity scores
    top_documents = df.iloc[top_indices]
    top_documents['similarity_score'] = similarities[0][top_indices]

    return top_documents[['filename', 'similarity_score', 'cleaned_content']]

# Example usage:
query = "Renovation of Oslo"
recommendations = recommend_documents(query, df, tokenizer, model, top_n=5)

# Display the recommendations
print(recommendations)


                                              filename  similarity_score  \
351  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          0.406282   
95   C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          0.404268   
246  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          0.404067   
316  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          0.403305   
107  C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...          0.402201   

                                       cleaned_content  
351  bilag kravspesifikasjon (oppdragsgivers tilbud...  
95   vefsn kommune avtaledokument revised 3/29/2022...  
246  konkurransegrunnlaget del konkurranseregler ra...  
316  bilag oppdragsgivers spesifikasjon omfang berg...  
107  larvik kommune virksomhet prosjekt bygg anlegg...  


C:\Users\spide\AppData\Local\Temp\ipykernel_17832\3011798110.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  document_embeddings = torch.stack([torch.tensor(embedding) for embedding in df['bert_embeddings'].values])
C:\Users\spide\AppData\Local\Temp\ipykernel_17832\3011798110.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_documents['similarity_score'] = similarities[0][top_indices]


In [29]:
import pandas as pd

# Load the CSV file containing the embeddings
file_path = r'C:\Users\spide\OneDrive\Desktop\Bachlorz\topics_with_embeddings.csv'
df = pd.read_csv(file_path)

# Keep only the relevant columns: 'filename', 'cleaned_content', and 'bert_embeddings'
df_relevant = df[['filename', 'cleaned_content', 'bert_embeddings']]

# Optionally, save the cleaned DataFrame to a new file
output_path = r'C:\Users\spide\OneDrive\Desktop\Bachlorz\cleaned_data_for_rag.csv'
df_relevant.to_csv(output_path, index=False)

# Check the result
print(f"Data saved to {output_path}")
df_relevant.head()  # Display the first few rows of the cleaned data


Data saved to C:\Users\spide\OneDrive\Desktop\Bachlorz\cleaned_data_for_rag.csv


,filename,cleaned_content,bert_embeddings
0,C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...,nedre romerike brann- redningsvesen iks revise...,[-1.24785826e-01 -2.81700402e-01 4.86699522e-...
1,C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...,unnamed: unnamed: unnamed: unnamed: unnamed: u...,[ 1.04157731e-01 -2.77466238e-01 1.50760800e-...
2,C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...,vedlegg ii.01 bilag seriøsitetskrav bygg- anle...,[-8.84428769e-02 -2.27621257e-01 4.16380584e-...
3,C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...,mal retningslinjer krav prøvedrift prosjekt: x...,[-0.46103162 -0.33935076 -0.13962997 0.725461...
4,C:\Users\spide\OneDrive\Desktop\Bachlorz\2022 ...,konkurransegrunnlag åpen anbudskonkurranse for...,[-2.13442773e-01 -4.13620234e-01 1.19351111e-...
